In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [2]:
# load vsech nasich + Honzovo dat o pozicich autobusu a spojeni do velke tabulky, odstaneni radku, kde je delay = NaN
data_nase = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)
data_wkend_nase = pd.read_csv('out_clean_vikend_16_17.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_nase, data_Honza, data_wkend,data_wkend_nase], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()

,UNICORN,TRIP_ID,START_TIME,HEADSIGN,DATE,BEARING,DELAY,DELAY_STOP_ARRIVAL,DELAY_STOP_DEPARTURE,IS_CANCELED,...,LAST_STOP_SEQUENCE,NEXT_STOP_ARRIVAL_TIME,NEXT_STOP_DEPARTURE_TIME,NEXT_STOP_ID,NEXT_STOP_SEQUENCE,ORIGIN_TIMESTAMP,SHAPE_DIST_TRAVELED,SPEED,LATITUDE,LONGITUDE
0,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,82,65.0,NaN,85.0,False,...,28.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:13:00 Z",U1564Z1,29.0,"Tue, 19 May 2020 19:13:17 Z",30.1,52,49.90354,14.58532
1,335_353_191111_19_May,335_353_191111,18:28:00,"Praha,Budějovická",19 May,313,-70.0,NaN,-32.0,False,...,26.0,"Tue, 19 May 2020 19:22:00 Z","Tue, 19 May 2020 19:17:00 Z",U488Z7,27.0,"Tue, 19 May 2020 19:15:06 Z",24.9,52,50.02853,14.45633
2,335_353_191111_19_May,335_353_191111,18:28:00,"Praha,Budějovická",19 May,358,-62.0,NaN,-57.0,False,...,27.0,"Tue, 19 May 2020 19:22:00 Z","Tue, 19 May 2020 19:22:00 Z",U50Z84,28.0,"Tue, 19 May 2020 19:16:13 Z",25.3,34,50.03116,14.45503
3,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,2,76.0,NaN,85.0,False,...,28.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:13:00 Z",U1564Z1,29.0,"Tue, 19 May 2020 19:12:54 Z",29.8,51,49.90253,14.58289
4,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,61,91.0,NaN,130.0,False,...,27.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:11:00 Z",U1565Z2,28.0,"Tue, 19 May 2020 19:11:59 Z",29.2,54,49.90037,14.57858


In [3]:
# pridani sloupce s route_name
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [4]:
# prevod sloupecku DATE na datovy format
data['DATE'] = data['DATE'] + " 2020"

In [5]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [6]:
# vytvoreni noveho sloupce se dnem v tydnu
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [7]:
# filtrace pouze linky 333, v ranni spicce, smer centrum -> novy mensi dataframe -> ranni_333
ranni_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') & (data['ROUTE_NAME'] == '333') & (data['START_TIME'] < '11:00:00'), ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
       'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
       'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
       'LONGITUDE','ROUTE_NAME','DAY_OF_WEEK']]

# pocet unikatnich spoju -> 195
len(ranni_333['UNICORN'].unique().tolist())

195

In [8]:
# pocet unikatnich trip_id v souboru 
len(ranni_333['TRIP_ID'].unique())

21

In [9]:
# DataFrame se vsemi jizdnimi rady (pro unikatnich 21 trip_id) -> vytvoreno v jizdni_rad.ipynb
info_stops_clean = pd.read_csv('jizdnirad.csv', encoding='utf-8', low_memory=False)

In [10]:
info_stops_clean

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,STOP.GEOMETRY.COORDINATES,STOP.PROPERTIES.STOP_ID,STOP.PROPERTIES.STOP_NAME
0,9:18:00,9:18:00,0.00000,U1503Z2,1,333_677_200302,"[14.45708, 49.96319]",U1503Z2,"Dolní Břežany,Obecní úřad"
1,9:19:00,9:19:00,0.15836,U1502Z2,2,333_677_200302,"[14.45889, 49.96373]",U1502Z2,"Dolní Břežany,Náměstí"
2,9:21:00,9:21:00,0.77992,U1501Z2,3,333_677_200302,"[14.46199, 49.96894]",U1501Z2,"Dolní Břežany,Na kopečku"
3,9:25:00,9:25:00,3.53721,U545Z2,4,333_677_200302,"[14.46853, 49.99163]",U545Z2,Písnice
4,9:26:00,9:26:00,3.91690,U546Z2,5,333_677_200302,"[14.46768, 49.99496]",U546Z2,Ke Březině
...,...,...,...,...,...,...,...,...,...
462,8:15:00,8:15:00,16.22898,U749Z2,24,333_1012_200307,"[14.45197, 50.02019]",U749Z2,Tempo
463,8:16:00,8:16:00,16.56781,U656Z2,25,333_1012_200307,"[14.45149, 50.02318]",U656Z2,Sídliště Krč
464,8:17:00,8:17:00,17.03180,U893Z4,26,333_1012_200307,"[14.45143, 50.02735]",U893Z4,Zálesí
465,8:19:00,8:19:00,17.58466,U488Z7,27,333_1012_200307,"[14.45515, 50.03038]",U488Z7,Nemocnice Krč


In [21]:
# vybrani sloupce trip_id s ujetymi km
stop_dist = info_stops_clean.loc[:,['TRIP_ID','STOP_ID','STOP_SEQUENCE','SHAPE_DIST_TRAVELED','DATE']]
stop_dist

,TRIP_ID,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED
0,333_677_200302,U1503Z2,1,0.00000
1,333_677_200302,U1502Z2,2,0.15836
2,333_677_200302,U1501Z2,3,0.77992
3,333_677_200302,U545Z2,4,3.53721
4,333_677_200302,U546Z2,5,3.91690
...,...,...,...,...
462,333_1012_200307,U749Z2,24,16.22898
463,333_1012_200307,U656Z2,25,16.56781
464,333_1012_200307,U893Z4,26,17.03180
465,333_1012_200307,U488Z7,27,17.58466


In [12]:
# info_stops_clean['NEJBLIZSI_POZICE'] = 0.0
# info_stops_clean['DELAY_STOP_DEPARTURE'] = 0.0
# info_stops_clean['DAY_OF_WEEK'] = ''
# info_stops_clean['DATE'] = ''

In [13]:
# list unikatnich unicornu z dataframe data (allpositions)
unicorn_list = ranni_333['UNICORN'].unique().tolist()
len(unicorn_list)

#unicorn_try = data[data['UNICORN'] == '333_1052_200302_06_May'].sort_values(by='SHAPE_DIST_TRAVELED')

195

In [111]:
# prazdny novy DataFrame
final_tab = pd.DataFrame([], 
                         columns = ['DAY_OF_WEEK','TRIP_ID','DATE','UNICORN','STOP_ID','STOP_SEQUENCE','SHAPE_DIST_TRAVELED','DELAY'])
final_tab

,DAY_OF_WEEK,TRIP_ID,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY


In [114]:
# 195 unicornu -> pro kazdeho unicorna vybrat jen sloupecky co me zajimaji
# najit ten spravny jizdni rad trip_id == trip_id
# a na jeho zaklade priradit (zkratit) pocet pozic jen na pocet zastavek
index = 0
final_tab = pd.DataFrame()
#                        --- TADY POZOR, projet všechny autobusy chvíli trvá tak jsem dal jen prvních deset..
for uni in unicorn_list[:10]:
    #tohle je jen pro vypisování stavu procesu
    print(round(index/len(unicorn_list)*100), '%')
    index+=1
    
    #vyberu vsechny pozice od daného spoje
    unicorn_find = ranni_333[ranni_333['UNICORN'] == uni]
    #vyberu jen sloupce co mě zajímají - možná zbytečené...
    bus_dist = unicorn_find.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','DELAY','DATE']]
    #tady si vytáhnu opakující se trip_id i trip_date
    gtfs_trip_id = bus_dist['TRIP_ID'].tolist()[0]
    trip_date = bus_dist['DATE'].tolist()[0]
    
    #pro každou zastávku v JŘ pro dané GTFS_TRIP_ID
    for i, stop in stop_dist[stop_dist['TRIP_ID'] == gtfs_trip_id].iterrows():
        
        #hledám nejbližší pozici tím, že si nastavím zatím nejbližší pozici jako hooodně daleko
        min_distance_so_far = 100000
        #i to zpoždění na nulu, ale prostě se vybere určitě alespoň jedna reálná
        stop_delay = 0
        for j, position in bus_dist.iterrows():
            
            # vypocita absolutni hodnotu rozdilu vzdalenosti
            actual_distance = abs(stop['SHAPE_DIST_TRAVELED'] - position['SHAPE_DIST_TRAVELED'])

            # a pokud je tahle pozice blíž než 100000 nebo blíž než poslední jiná nalezená
            if (min_distance_so_far > actual_distance):
                #tak si uližím aktuální nejbližší vzdálenost
                min_distance_so_far = actual_distance
                #a uložím si k ní i její zpoždění
                stop_delay = position['DELAY']

        # teď už mi to projelo k dané zastávce všema pozicema a snad jsem nalezl opravdu tu nejbližší
        #udělám si výsledný dataframe se všemi potřebnými údaji
        df = pd.DataFrame(
            {
                "TRIP_ID": [gtfs_trip_id], 
                "DAY_OF_WEEK": [gtfs_trip_id],
                "DATE": [trip_date],
                "UNICORN": [uni],
                "STOP_ID": [stop['STOP_ID']],
                "STOP_SEQUENCE": [stop['STOP_SEQUENCE']],
                "SHAPE_DIST_TRAVELED": [stop['SHAPE_DIST_TRAVELED']],
                "DELAY": [stop_delay],
            }
        )
        # a uložím do jednoho velkého
        final_tab = final_tab.append(df, ignore_index = True)
        

0 %
1 %
1 %
2 %
2 %
3 %
3 %
4 %
4 %
5 %


In [128]:
#tohle jsou všechny trip_id a podíváme se
final_tab['TRIP_ID'].unique()

array(['333_677_200302', '333_1072_200302', '333_557_200302',
       '333_1064_200302', '333_1115_200302', '333_1012_200302',
       '333_1052_200302', '333_1094_200302', '333_715_200302',
       '333_751_200302'], dtype=object)

In [126]:
# jak vypadá trasa tohodle
final_tab[final_tab['TRIP_ID']=='333_1115_200302']

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY
93,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1513Z2,1,0.00000,17.0
94,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U6955Z2,2,0.75810,67.0
95,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1512Z2,3,1.23504,58.0
96,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1507Z1,4,1.58509,52.0
97,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1508Z1,5,2.51503,54.0
98,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1509Z1,6,3.14104,93.0
99,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1508Z2,7,3.77136,77.0
100,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1507Z2,8,4.75176,24.0
101,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1506Z2,9,5.24994,-4.0
102,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1504Z1,10,6.49131,17.0


In [117]:
# a jak tohodle
final_tab[final_tab['TRIP_ID']=='333_751_200302']

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY
217,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1503Z2,1,0.00000,40.0
218,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1502Z2,2,0.15836,39.0
219,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1501Z2,3,0.77992,-1.0
220,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U545Z2,4,3.53721,-6.0
221,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U546Z2,5,3.91690,-22.0
222,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U871Z2,6,4.11683,-32.0
223,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U369Z2,7,4.61837,-15.0
224,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U783Z2,8,5.09108,26.0
225,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U335Z2,9,5.46109,114.0
226,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U804Z2,10,5.77175,95.0


In [129]:
#no je vidět že první má 26 zastávek a druhý 16, ale ten první od zastávky 11 jede stejně jako ten druhý...
# úkol je to teď nějak zkonsolidovat...


In [131]:
final_tab[final_tab['TRIP_ID']=='333_1115_200302'][10:]

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY
103,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1503Z2,11,8.51563,-43.0
104,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1502Z2,12,8.67399,-15.0
105,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U1501Z2,13,9.29555,-45.0
106,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U545Z2,14,12.05284,-29.0
107,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U546Z2,15,12.43253,-16.0
108,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U871Z2,16,12.63246,-45.0
109,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U369Z2,17,13.13400,-11.0
110,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U783Z2,18,13.60671,22.0
111,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U335Z2,19,13.97672,22.0
112,333_1115_200302,333_1115_200302,2020-05-20,333_1115_200302_20_May,U804Z2,20,14.28738,28.0


In [132]:
final_tab[final_tab['TRIP_ID']=='333_751_200302'][:]

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY
217,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1503Z2,1,0.00000,40.0
218,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1502Z2,2,0.15836,39.0
219,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U1501Z2,3,0.77992,-1.0
220,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U545Z2,4,3.53721,-6.0
221,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U546Z2,5,3.91690,-22.0
222,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U871Z2,6,4.11683,-32.0
223,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U369Z2,7,4.61837,-15.0
224,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U783Z2,8,5.09108,26.0
225,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U335Z2,9,5.46109,114.0
226,333_751_200302,333_751_200302,2020-05-20,333_751_200302_20_May,U804Z2,10,5.77175,95.0


In [ ]:
#co třeba takto? :-)